[**Blueprints for Text Analysis Using Python**](https://github.com/blueprints-for-text-analytics-python/blueprints-text)  
Jens Albrecht, Sidharth Ramachandran, Christian Winkler

**If you like the book or the code examples here, please leave a friendly comment on [Amazon.com](https://www.amazon.com/Blueprints-Text-Analytics-Using-Python/dp/149207408X)!**
<img src="../rating.png" width="100"/>


# Chapter 9:<div class='tocSkip'/>

## Remark<div class='tocSkip'/>

The code in this notebook differs slightly from the printed book. For example we frequently use pretty print (`pp.pprint`) instead of `print` and `tqdm`'s `progress_apply` instead of Pandas' `apply`. 

Moreover, several layout and formatting commands, like `figsize` to control figure size or subplot commands are removed in the book.

You may also find some lines marked with three hashes ###. Those are not in the book as well as they don't contribute to the concept.

All of this is done to simplify the code in the book and put the focus on the important parts instead of formatting.

## Setup<div class='tocSkip'/>

Set directory locations. If working on Google Colab: copy files and install required libraries.

In [1]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master'
    os.system(f'wget {GIT_ROOT}/ch09/setup.py')

%run -i setup.py

You are working on a local system.
Files will be searched relative to "..".


## Load Python Settings<div class="tocSkip"/>

Common imports, defaults for formatting in Matplotlib, Pandas etc.

In [2]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# path to import blueprints packages
sys.path.append(BASE_DIR + '/packages')

In [3]:
# adjust matplotlib resolution for book version
matplotlib.rcParams.update({'figure.dpi': 200 })

In [4]:
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
import matplotlib.pyplot as plt
import html
import re
import random
import rouge_score
import wikipediaapi

# How to create short summaries of long textual information for quick reading

# Text Summarization

## Extractive Methods


## Data pre-processing 


In [4]:
import requests
from bs4 import BeautifulSoup
import os.path
from dateutil import parser
import pandas as pd
import numpy as np

In [5]:
def download_article(url):
    # check if article already there
    filename = url.split("/")[-1] + ".html"
    filename = f"{BASE_DIR}/ch09/" + filename
    if not os.path.isfile(filename):
        r = requests.get(url)
        with open(filename, "w+") as f:
            f.write(r.text)
    return filename

In [6]:
def parse_article(article_file):
    with open(article_file, "r") as f:
        html = f.read()
    r = {}
    soup = BeautifulSoup(html, 'html.parser')
    r['id'] = soup.select_one("div.StandardArticle_inner-container")['id']
    r['url'] = soup.find("link", {'rel': 'canonical'})['href']
    r['headline'] = soup.h1.text
    r['section'] = soup.select_one("div.ArticleHeader_channel a").text
    
    r['text'] = soup.select_one("div.StandardArticleBody_body").text
    r['authors'] = [a.text 
                    for a in soup.select("div.BylineBar_first-container.ArticleHeader_byline-bar\
                                          div.BylineBar_byline span")]
    r['time'] = soup.find("meta", { 'property': "og:article:published_time"})['content']
    return r

In [7]:
import reprlib
r = reprlib.Repr()
r.maxstring = 800

url1 = "https://www.reuters.com/article/us-qualcomm-m-a-broadcom-5g/what-is-5g-and-who-are-the-major-players-idUSKCN1GR1IN"
article_name1 = download_article(url1)
article1 = parse_article(article_name1)
print ('Article Published on', r.repr(article1['time']))
print (r.repr(article1['text']))

Article Published on '2018-03-15T11:36:28+0000'
'LONDON/SAN FRANCISCO (Reuters) - U.S. President Donald Trump has blocked microchip maker Broadcom Ltd’s (AVGO.O) $117 billion takeover of rival Qualcomm (QCOM.O) amid concerns that it would give China the upper hand in the next generation of mobile communications, or 5G. A 5G sign is seen at the Mobile World Congress in Barcelona, Spain February 28, 2018. REUTERS/Yves HermanBelow are some facts... 4G wireless and looks set to top the list of patent holders heading into the 5G cycle. Huawei, Nokia, Ericsson and others are also vying to amass 5G patents, which has helped spur complex cross-licensing agreements like the deal struck late last year Nokia and Huawei around handsets. Editing by Kim Miyoung in Singapore and Jason Neely in LondonOur Standards:The Thomson Reuters Trust Principles.'


# Blueprint - Summarizing text using topic representation

## Identifying important words with TF-IDF values


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize

sentences = tokenize.sent_tokenize(article1['text'])
tfidfVectorizer = TfidfVectorizer()
words_tfidf = tfidfVectorizer.fit_transform(sentences)

In [34]:
# Parameter to specify number of summary sentences required
num_summary_sentence = 3

# Sort the sentences in descending order by the sum of TF-IDF values
sent_sum = words_tfidf.sum(axis=1)
important_sent = np.argsort(sent_sum, axis=0)[::-1]

# Print three most important sentences in the order they appear in the article
for i in range(0, len(sentences)):
    if i in important_sent[:num_summary_sentence]:
        print (sentences[i])

LONDON/SAN FRANCISCO (Reuters) - U.S. President Donald Trump has blocked microchip maker Broadcom Ltd’s (AVGO.O) $117 billion takeover of rival Qualcomm (QCOM.O) amid concerns that it would give China the upper hand in the next generation of mobile communications, or 5G.
5G networks, now in the final testing stage, will rely on denser arrays of small antennas and the cloud to offer data speeds up to 50 or 100 times faster than current 4G networks and serve as critical infrastructure for a range of industries.
The concern is that a takeover by Singapore-based Broadcom could see the firm cut research and development spending by Qualcomm or hive off strategically important parts of the company to other buyers, including in China, U.S. officials and analysts have said.


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize

def tfidf_summary(text, num_summary_sentence):
    summary_sentence = []
    sentences = tokenize.sent_tokenize(text)
    tfidfVectorizer = TfidfVectorizer()
    words_tfidf = tfidfVectorizer.fit_transform(sentences)
    sentence_sum = words_tfidf.sum(axis=1)
    important_sentences = np.argsort(sentence_sum, axis=0)[::-1]
    for i in range(0, len(sentences)):
        if i in important_sentences[:num_summary_sentence]:
            summary_sentence.append(sentences[i])
    return summary_sentence

## LSA algorithm


In [36]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

from sumy.summarizers.lsa import LsaSummarizer

LANGUAGE = "english"
stemmer = Stemmer(LANGUAGE)

parser = PlaintextParser.from_string(article1['text'], Tokenizer(LANGUAGE))
summarizer = LsaSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, num_summary_sentence):
    print (str(sentence))

LONDON/SAN FRANCISCO (Reuters) - U.S. President Donald Trump has blocked microchip maker Broadcom Ltd’s (AVGO.O) $117 billion takeover of rival Qualcomm (QCOM.O) amid concerns that it would give China the upper hand in the next generation of mobile communications, or 5G.
Moving to new networks promises to enable new mobile services and even whole new business models, but could pose challenges for countries and industries unprepared to invest in the transition.
The concern is that a takeover by Singapore-based Broadcom could see the firm cut research and development spending by Qualcomm or hive off strategically important parts of the company to other buyers, including in China, U.S. officials and analysts have said.


In [37]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.lsa import LsaSummarizer

def lsa_summary(text, num_summary_sentence):
    summary_sentence = []
    LANGUAGE = "english"
    stemmer = Stemmer(LANGUAGE)
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    summarizer = LsaSummarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, num_summary_sentence):
        summary_sentence.append(str(sentence))
    return summary_sentence

In [38]:
r.maxstring = 800
url2 = "https://www.reuters.com/article/us-usa-economy-watchlist-graphic/predicting-the-next-u-s-recession-idUSKCN1V31JE"
article_name2 = download_article(url2)
article2 = parse_article(article_name2)
print ('Article Published', r.repr(article1['time']))
print (r.repr(article2['text']))

Article Published '2018-03-15T11:36:28+0000'
'NEW YORK A protracted trade war between China and the United States, the world’s largest economies, and a deteriorating global growth outlook has left investors apprehensive about the end to the longest expansion in American history. FILE PHOTO: Ships and shipping containers are pictured at the port of Long Beach in Long Beach, California, U.S., January 30, 2019.   REUTERS/Mike BlakeThe recent ...hton wrote in the June Cass Freight Index report.  12. MISERY INDEX The so-called Misery Index adds together the unemployment rate and the inflation rate. It typically rises during recessions and sometimes prior to downturns. It has slipped lower in 2019 and does not look very miserable.  Reporting by Saqib Iqbal Ahmed; Editing by Chizu NomiyamaOur Standards:The Thomson Reuters Trust Principles.'


In [39]:
summary_sentence = tfidf_summary(article2['text'], num_summary_sentence)
for sentence in summary_sentence:
    print (sentence)

REUTERS/Mike BlakeThe recent rise in U.S.-China trade war tensions has brought forward the next U.S. recession, according to a majority of economists polled by Reuters who now expect the Federal Reserve to cut rates again in September and once more next year.
On Tuesday, U.S. stocks jumped sharply higher and safe-havens like the Japanese yen and Gold retreated after the U.S. Trade Representative said additional tariffs on some Chinese goods, including cell phones and laptops, will be delayed to Dec. 15.
ISM said its index of national factory activity slipped to 51.2 last month, the lowest reading since August 2016, as U.S. manufacturing activity slowed to a near three-year low in July and hiring at factories shifted into lower gear, suggesting a further loss of momentum in economic growth early in the third quarter.


In [40]:
summary_sentence = lsa_summary(article2['text'], num_summary_sentence)
for sentence in summary_sentence:
    print (sentence)

NEW YORK A protracted trade war between China and the United States, the world’s largest economies, and a deteriorating global growth outlook has left investors apprehensive about the end to the longest expansion in American history.
REUTERS/Mike BlakeThe recent rise in U.S.-China trade war tensions has brought forward the next U.S. recession, according to a majority of economists polled by Reuters who now expect the Federal Reserve to cut rates again in September and once more next year.
Trade tensions have pulled corporate confidence and global growth to multi-year lows and U.S. President Donald Trump’s announcement of more tariffs have raised downside risks significantly, Morgan Stanley analysts said in a recent note.


# Blueprint - Summarizing text using an indicator representation

In [41]:
from sumy.summarizers.text_rank import TextRankSummarizer

parser = PlaintextParser.from_string(article2['text'], Tokenizer(LANGUAGE))
summarizer = TextRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, num_summary_sentence):
    print (str(sentence))

REUTERS/Mike BlakeThe recent rise in U.S.-China trade war tensions has brought forward the next U.S. recession, according to a majority of economists polled by Reuters who now expect the Federal Reserve to cut rates again in September and once more next year.
As recession signals go, this so-called inversion in the yield curve has a solid track record as a predictor of recessions.
Markets turned down before the 2001 recession and tumbled at the start of the 2008 recession.


In [42]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.text_rank import TextRankSummarizer

def textrank_summary(text, num_summary_sentence):
    summary_sentence = []
    LANGUAGE = "english"
    stemmer = Stemmer(LANGUAGE)
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    summarizer = TextRankSummarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, num_summary_sentence):
        summary_sentence.append(str(sentence))
    return summary_sentence

In [43]:
parser = PlaintextParser.from_string(article1['text'], Tokenizer(LANGUAGE))
summarizer = TextRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, num_summary_sentence):
    print (str(sentence))

Acquiring Qualcomm would represent the jewel in the crown of Broadcom’s portfolio of communications chips, which supply wi-fi, power management, video and other features in smartphones alongside Qualcomm’s core baseband chips - radio modems that wirelessly connect phones to networks.
Qualcomm (QCOM.O) is the dominant player in smartphone communications chips, making half of all core baseband radio chips in smartphones.
Slideshow (2 Images)The standards are set by a global body to ensure all phones work across different mobile networks, and whoever’s essential patents end up making it into the standard stands to reap huge royalty licensing revenue streams.


In [44]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

In [45]:
r.maxstring = 500

In [46]:
p_wiki = wiki_wiki.page('Mongol_invasion_of_Europe')
print (r.repr(p_wiki.text))

'The Mongol invasion of Europe in the 13th century occurred from the 1220s into the 1240s. In Eastern Europe, the Mongols destroyed Volga Bulgaria, Cumania, Alania, and the Kievan Rus\' federation. In Central Europe, the Mongol armies launched a tw...tnotes\nReferences\nSverdrup, Carl (2010). "Numbers in Mongol Warfare". Journal of Medieval Military History. Boydell Press. 8: 109–17 [p. 115]. ISBN 978-1-84383-596-7.\n\nFurther reading\nExternal links\nThe Islamic World to 1600: The Golden Horde'


In [47]:
r.maxstring = 200

num_summary_sentence = 10

summary_sentence = textrank_summary(p_wiki.text, num_summary_sentence)

for sentence in summary_sentence:
    print (sentence)

In Central Europe, the Mongol armies launched a two-pronged invasion of fragmented Poland, culminating in the Battle of Legnica (9 April 1241), and the Kingdom of Hungary, culminating in the Battle of Mohi (11 April 1241).
Warring European princes realized they had to cooperate in the face of a Mongol invasion, so local wars and conflicts were suspended in parts of central Europe, only to be resumed after the Mongols had withdrawn.
Under Wenceslaus' leadership during the Mongol invasion, Bohemia remained one of a few European kingdoms that was never conquered and molested by the Mongols even though most kingdoms around it such as Poland and Moravia were ravaged.
A major Hungarian loss was imminent, and the Mongols intentionally left a gap in their formation to permit the wavering Hungarian forces to flee and spread out in doing so, leaving them unable to effectively resist the Mongols as they picked off the retreating Hungarian remnants.
European tactics against Mongols The traditional

In [49]:
parser = PlaintextParser.from_file(f"{BASE_DIR}/ch09/acl2017.tex", Tokenizer(LANGUAGE))
summarizer = TextRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, 5):
    print (str(sentence))

In more detail, our contributions are as follows: \begin{itemize}[noitemsep] \item{We introduce a new dataset for summarisation of scientific publications consisting of over 10k documents} \item{Following the approach of \cite{hermann2015teaching} in the news domain, we introduce a method, \textit{HighlightROUGE}, which can be used to automatically extend this dataset %extractive summarisation datasets% and show empirically that this improves summarisation performance} \item{Taking inspiration from previous work in summarising scientific literature \citep{kupiec1995trainable, papers_citationSaggion2016}, we introduce a %further metric we use as a feature, \textit{AbstractROUGE}, which can be used to extract summaries by exploiting the abstract of a paper} \item{We benchmark several neural as well traditional summarisation methods on the dataset and use simple features to model the global context of a summary statement, which contribute most to the overall score} \item{We compare our be

# Measuring the performance of Text Summarization methods

In [50]:
def print_rouge_score(rouge_score):
    for k,v in rouge_score.items():
        print (k, 'Precision:', "{:.2f}".format(v.precision), 'Recall:', "{:.2f}".format(v.recall), 'fmeasure:', "{:.2f}".format(v.fmeasure))

In [51]:
num_summary_sentence = 3 ##
gold_standard = article2['headline']
summary = ""

summary = ''.join(textrank_summary(article2['text'], num_summary_sentence))
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
scores = scorer.score(gold_standard, summary)
print_rouge_score(scores)

rouge1 Precision: 0.06 Recall: 0.83 fmeasure: 0.11


In [52]:
summary = ''.join(lsa_summary(article2['text'], num_summary_sentence))
scores = scorer.score(gold_standard, summary)
print_rouge_score(scores)

rouge1 Precision: 0.04 Recall: 0.83 fmeasure: 0.08


In [53]:
num_summary_sentence = 10 ##
gold_standard = p_wiki.summary

summary = ''.join(textrank_summary(p_wiki.text, num_summary_sentence))

scorer = rouge_scorer.RougeScorer(['rouge2','rougeL'], use_stemmer=True)
scores = scorer.score(gold_standard, summary)
print_rouge_score(scores)

rouge2 Precision: 0.18 Recall: 0.46 fmeasure: 0.26
rougeL Precision: 0.16 Recall: 0.40 fmeasure: 0.23


In [54]:
summary = ''.join(lsa_summary(p_wiki.text, num_summary_sentence))

scorer = rouge_scorer.RougeScorer(['rouge2','rougeL'], use_stemmer=True)
scores = scorer.score(gold_standard, summary)
print_rouge_score(scores)

rouge2 Precision: 0.04 Recall: 0.08 fmeasure: 0.05
rougeL Precision: 0.12 Recall: 0.25 fmeasure: 0.16


# Blueprint - Summarizing text using machine learning

## Step 1 - Creating target labels


In [5]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows',1000)

file = "travel_threads.csv.gz"
file = f"{BASE_DIR}/data/travel-forum-threads/travel_threads.csv.gz" ### real location
df = pd.read_csv(file, sep='|', dtype={'ThreadID': 'object'})
df[df['ThreadID']=='60763_5_3122150'].head(1).T

# You can view the actual post here ###
# URL - https://www.tripadvisor.com/ShowTopic-g60763-i5-k3122150-Which_attractions_need_to_be_pre_booked-New_York_City_New_York.html ###

,850
Filename,60763_5_3122150
ThreadID,60763_5_3122150
Title,which attractions need to be pre booked?
userID,musicqueenLon...
Date,"29 September 2009, 1:41"
postNum,1
text,Hi I am coming to NY in Oct! So excited&quot; Have wanted to visit for years. We are planning on doing all the usual stuff so wont list it all but wondered which attractions should be pre booked and which can you just turn up at> I am plannin on booking ESB but what else? thanks x
summary,A woman was planning to travel NYC in October and needed some suggestions about attractions in the NYC. She was planning on booking ESB.Someone suggested that the TOTR was much better compared to ESB. The other suggestion was to prebook the show to avoid wasting time in line.Someone also suggested her New York Party Shuttle tours.


In [6]:
# Cleaning up the data to remove special characters
# Re-using the blueprint from Chapter 4
# Renamed as regex_clean to differentiate with the next clean function
from blueprints.preparation import clean as regex_clean

In [7]:
# Re-using the blueprint from Chapter 4 but adapting to add additional steps specific to this dataset
import re ###
import spacy ###
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, \
                       compile_infix_regex, compile_suffix_regex
from textacy.preprocessing.replace import replace_urls
import textacy

def custom_tokenizer(nlp):
    
    # use default patterns except the ones matched by re.search
    prefixes = [pattern for pattern in nlp.Defaults.prefixes 
                if pattern not in ['-', '_', '#']]
    suffixes = [pattern for pattern in nlp.Defaults.suffixes
                if pattern not in ['_']]
    infixes  = [pattern for pattern in nlp.Defaults.infixes
                if not re.search(pattern, 'xx-xx')]

    return Tokenizer(vocab          = nlp.vocab, 
                     rules          = nlp.Defaults.tokenizer_exceptions,
                     prefix_search  = compile_prefix_regex(prefixes).search,
                     suffix_search  = compile_suffix_regex(suffixes).search,
                     infix_finditer = compile_infix_regex(infixes).finditer,
                     token_match    = nlp.Defaults.token_match)

nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = custom_tokenizer(nlp)

def extract_lemmas(doc, **kwargs):
    return [t.lemma_ for t in textacy.extract.words(doc, **kwargs)]

def extract_noun_chunks(doc, include_pos=['NOUN'], sep='_'):
    chunks = []
    for noun_chunk in doc.noun_chunks:
        chunk = [token.lemma_ for token in noun_chunk
                 if token.pos_ in include_pos]
        if len(chunk) >= 2:
            chunks.append(sep.join(chunk))
    return chunks

def extract_entities(doc, include_types=None, sep='_'):

    ents = textacy.extract.entities(doc, 
             include_types=include_types, 
             exclude_types=None, 
             drop_determiners=True, 
             min_freq=1)
    
    return [re.sub('\s+', sep, e.lemma_)+'/'+e.label_ for e in ents]

def clean(text):
    # Replace URLs
    text = replace_urls(text)
    
    # Replace semi-colons (relevant in Java code ending)
    text = text.replace(';','')
    
    # Replace character tabs (present as literal in description field)
    text = text.replace('\t','')
    
    # Find and remove any stack traces - doesn't fix all code fragments but removes many exceptions
    start_loc = text.find("Stack trace:")
    text = text[:start_loc]
    
    # Remove Hex Code
    text = re.sub(r'(\w+)0x\w+', '', text)
    
    # Initialize Spacy
    doc = nlp(text)
    
    # From Blueprint function
    lemmas          = extract_lemmas(doc, 
                                     exclude_pos = ['PART', 'PUNCT', 'DET', 'PRON', 'SYM', 'SPACE', 'NUM'],
                                     filter_stops = True,
                                     filter_nums = True,
                                     filter_punct = True)

    return lemmas

In [8]:
# Applying regex based cleaning function
df['text'] = df['text'].apply(regex_clean)
# Extracting lemmas using spacy pipeline
df['lemmas'] = df['text'].apply(clean)

In [85]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_split, test_split = next(gss.split(df, groups=df['ThreadID']))

In [86]:
train_df = df.iloc[train_split]
test_df = df.iloc[test_split]

print ('Number of threads for Training ', train_df['ThreadID'].nunique())
print ('Number of threads for Testing ', test_df['ThreadID'].nunique())

Number of threads for Training  559
Number of threads for Testing  140


In [87]:
import textdistance

compression_factor = 0.3

train_df['similarity'] = train_df.apply(
    lambda x: textdistance.jaro_winkler(x.text, x.summary), axis=1)
train_df["rank"] = train_df.groupby("ThreadID")["similarity"].rank(
    "max", ascending=False)

topN = lambda x: x <= np.ceil(compression_factor * x.max())
train_df['summaryPost'] = train_df.groupby('ThreadID')['rank'].apply(topN)

In [89]:
train_df[['text','summaryPost']][train_df['ThreadID']=='60763_5_3122150'].head(3)

,text,summaryPost
850,"Hi I am coming to NY in Oct! So excited"" Have wanted to visit for years. We are planning on doing all the usual stuff so wont list it all but wondered which attractions should be pre booked and which can you just turn up at> I am plannin on booking ESB but what else? thanks x",True
851,I wouldnt bother doing the ESB if I was you TOTR is much better. What other attractions do you have in mind?,False
852,"The Statue of Liberty, if you plan on going to the statue itself or to Ellis Island (as opposed to taking a boat past): http://www.statuecruises.com/ Also, we prefer to book shows and plays in advance rather than trying for the same-day tickets, as that allows us to avoid wasting time in line. If that sounds appealing to you, have a look at http://www.broadwaybox.com/",True


## Step 2 - Adding features to assist model prediction


In [90]:
train_df['titleSimilarity'] = train_df.apply(
    lambda x: textdistance.jaro_winkler(x.text, x.Title), axis=1)

In [91]:
## Adding post length as a feature
train_df['textLength'] = train_df['text'].str.len()

In [92]:
train_df.loc[train_df['textLength'] <= 20, 'summaryPost'] = False

In [93]:
feature_cols = ['titleSimilarity','textLength','postNum']

In [94]:
train_df['combined'] = [
    ' '.join(map(str, l)) for l in train_df['lemmas'] if l is not '']
tfidf = TfidfVectorizer(min_df=10, ngram_range=(1, 2), stop_words="english")
tfidf_result = tfidf.fit_transform(train_df['combined']).toarray()

tfidf_df = pd.DataFrame(tfidf_result, columns=tfidf.get_feature_names_out())
tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
tfidf_df.index = train_df.index
train_df_tf = pd.concat([train_df[feature_cols], tfidf_df], axis=1)

In [88]:
test_df['similarity'] = test_df.apply(lambda x: textdistance.jaro_winkler(x.text, x.summary), axis=1)
test_df["rank"] = test_df.groupby("ThreadID")["similarity"].rank("max", ascending=False)

topN = lambda x: x <= np.ceil(compression_factor * x.max())
test_df['summaryPost'] = test_df.groupby('ThreadID')['rank'].apply(topN)

test_df['titleSimilarity'] = test_df.apply(lambda x: textdistance.jaro_winkler(x.text, x.Title), axis=1)

test_df['textLength'] = test_df['text'].str.len()

test_df.loc[test_df['textLength'] <= 20, 'summaryPost'] = False

test_df['combined'] = [' '.join(map(str, l)) for l in test_df['lemmas'] if l is not '']

tfidf_result = tfidf.transform(test_df['combined']).toarray()

tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names_out())
tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
tfidf_df.index = test_df.index
test_df_tf = pd.concat([test_df[feature_cols], tfidf_df], axis=1)

## Step 3 - Build a machine learning model


In [95]:
from sklearn.ensemble import RandomForestClassifier

model1 = RandomForestClassifier(random_state=20)
model1.fit(train_df_tf, train_df['summaryPost'])

RandomForestClassifier(random_state=20)

In [96]:
# Function to calculate rouge_score for each thread
def calculate_rouge_score(x, column_name):
    # Get the original summary - only first value since they are repeated
    ref_summary = x['summary'].values[0]
    
    # Join all posts that have been predicted as summary
    predicted_summary = ''.join(x['text'][x[column_name]])
    
    # Return the rouge score for each ThreadID
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = scorer.score(ref_summary, predicted_summary)
    return scores['rouge1'].fmeasure

In [97]:
test_df['predictedSummaryPost'] = model1.predict(test_df_tf)
print('Mean ROUGE-1 Score for test threads',
      test_df.groupby('ThreadID')[['summary','text','predictedSummaryPost']] \
      .apply(calculate_rouge_score, column_name='predictedSummaryPost').mean())

Mean ROUGE-1 Score for test threads 0.3454255443194838


In [98]:
random.seed(2)
random.sample(test_df['ThreadID'].unique().tolist(), 1)

['60763_5_3139646']

In [99]:
example_df = test_df[test_df['ThreadID'] == '60974_588_2180141']
print('Total number of posts', example_df['postNum'].max())
print('Number of summary posts',
      example_df[example_df['predictedSummaryPost']].count().values[0])
print('Title: ', example_df['Title'].values[0])
example_df[['postNum', 'text']][example_df['predictedSummaryPost']]

Total number of posts 9
Number of summary posts 2
Title:  What's fun for kids?


,postNum,text
813,4,"Well, you're really in luck, because there's a lot going on, including the Elmwood Avenue Festival of the Arts (http://www.elmwoodartfest.org), with special activities for youngsters, performances (including one by Nikki Hicks, one of my favorite local vocalists), and food of all kinds. Elmwood Avenue is one of the area's most colorful and thriving neighborhoods, and very walkable. The Buffalo Irish Festival is also going on that weekend in Hamburg, as it happens, at the fairgrounds: www.buf..."
814,5,"Depending on your time frame, a quick trip to Niagara Falls would be great. It is a 45 minute drive from Hamburg and well worth the investment of time. Otherwise you have some beaches in Angola to enjoy. If the girls like to shop you have the Galleria, which is a great expansive Mall. If you enjoy a more eclectic afternoon, lunch on Elmwood Avenue, a stroll through the Albright Know Art gallery, and hitting some of the hip shops would be a cool afternoon. Darien Lake Theme Park is 40 minutes..."


# Closing Remarks

# Further reading
